In [1]:
import pandas as pd

In [3]:
X_train = pd.read_csv("x_train.csv")
X_test = pd.read_csv("x_test.csv")
y_train = pd.read_csv("y_train.csv")

In [4]:
print(X_train.describe())

                  ID           DATE          STOCK       INDUSTRY  \
count  418595.000000  418595.000000  418595.000000  418595.000000   
mean   209297.000000     108.659002    3373.567833      37.176020   
std    120838.112303      61.891642    1533.157749      19.706505   
min         0.000000       0.000000       0.000000       0.000000   
25%    104648.500000      56.000000    2478.000000      22.000000   
50%    209297.000000     104.000000    3560.000000      43.000000   
75%    313945.500000     161.000000    4606.000000      53.000000   
max    418594.000000     223.000000    5716.000000      74.000000   

       INDUSTRY_GROUP         SECTOR   SUB_INDUSTRY          RET_1  \
count   418595.000000  418595.000000  418595.000000  416236.000000   
mean        12.697959       5.483845      90.391663       0.001383   
std          7.231701       2.410113      47.491157       0.031311   
min          0.000000       0.000000       0.000000      -0.845324   
25%          6.000000       